In [18]:
import pandas as pd


In [24]:
# Load the data
df = pd.read_csv(
    filepath_or_buffer="../data/trump_data.csv",
    dtype={"trump": int}
)

# Convert specific columns to lists of integers
df['hand'] = df['hand'].apply(lambda x: [int(i) for i in x.replace("[", "").replace("]", "").split(",")])
df['score'] = df['score'].apply(lambda x: [int(i) for i in x.replace("[", "").replace("]", "").split(",")])

df["score_avg"] = df["score"].apply(lambda x: sum(x) / len(x))
df["score_median"] = df["score"].apply(lambda x: sorted(x)[len(x) // 2])
df["score_06"] = df["score"].apply(lambda x: sorted(x)[int(len(x) * 0.6)])
df["score_07"] = df["score"].apply(lambda x: sorted(x)[int(len(x) * 0.7)])
df["score_08"] = df["score"].apply(lambda x: sorted(x)[int(len(x) * 0.8)])
df["score_09"] = df["score"].apply(lambda x: sorted(x)[int(len(x) * 0.9)])

df["hand_id"] = df["hand"].apply(lambda hand: int(''.join(str(x) for x in hand), 2))

In [26]:
unique_hands = df["hand_id"].unique()
best_trump = {
    hand: df[df["hand_id"] == hand].loc[df[df["hand_id"] == hand]["score_avg"].idxmax()]["trump"]
    for hand in unique_hands
}

# get best trump per unique hand 
df["best_trump"] = df["hand_id"].apply(lambda x: best_trump[x])

df.head(1000)

6000

In [21]:
# train, validation, test split unique hands
split = [int(len(unique_hands) * 0.8), int(len(unique_hands) * 0.9)]

train_hands = unique_hands[:split[0]]
validation_hands = unique_hands[split[0]:split[1]]
test_hands = unique_hands[split[1]:]

train_df = df[df["hand_id"].isin(train_hands)][["hand", "best_trump"]]
validation_df = df[df["hand_id"].isin(validation_hands)][["hand", "best_trump"]]
test_df = df[df["hand_id"].isin(test_hands)][["hand", "best_trump"]]

In [22]:
import torch
from torch import nn
import torch.nn.Module as Module


class TrumpSelectorNN(Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(36, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 6)
        )

    def forward(self, x):
        return self.fc(x)


def train():
    model = TrumpSelectorNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    for epoch in range(1000):
        print(f"Epoch: {epoch}")
        optimizer.zero_grad()
        output = model(train_df["hand"].apply(lambda x: torch.tensor(x).float()).tolist())
        loss = criterion(output, torch.tensor(train_df["best_trump"]).long())
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f"Epoch: {epoch}, Loss: {loss.item()}")

    return model

OSError: [WinError 126] The specified module could not be found. Error loading "C:\code\dl4g\.venv\lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

# Unfinished and wont be developed further